In [ ]:
import numpy as np
import tensorflow as tf
import os

batch_size=32
img_height=64
img_width=64

base_dir = os.path.join(os.getcwd())
train_dir = os.path.join(base_dir, 'asl_alphabet_train/asl_alphabet_train')

print(base_dir)


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.1,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
validation_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.20,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
test_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split=0.8,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
import matplotlib.pyplot as plt

class_names = train_ds.class_names
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(len(class_names)):
        ax = plt.subplot(6,5 , i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
from keras import Sequential
from keras import layers

model = Sequential([
    layers.Rescaling(1./255),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),

    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    history = model.fit(train_ds, batch_size=32,validation_batch_size=32, validation_data=validation_ds,epochs=10)
else: 
    with tf.device('/device:GPU:0'):
        history = model.fit(train_ds, batch_size=32,validation_batch_size=32, validation_data=validation_ds,epochs=10)


In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'r', label='Training Loss')
plt.plot(epochs, val_loss, 'b', label='Validation Loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
model.save('model1.h5')

In [ ]:
actual = []
pred = []
for images, labels in test_ds:
    for i in range(0, len(images)):
        image = images[i]
        image = np.expand_dims(image, axis=0)
        result = model.predict(image)
        pred.append(class_names[np.argmax(result)])
        actual.append(class_names[labels[i].numpy()])

In [ ]:
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.figure(figsize=(15, 15))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true=actual, y_pred=pred)
plot_confusion_matrix(cm=cm, classes=class_names, title='Confusion Matrix')

In [ ]:
# from keras.preprocessing import image

# actual = []
# pred = []
# base_dir = os.getcwd()
# directories = os.listdir(base_dir + '\..\input\asl-alphabet-test\\')
# for directory in directories:
#     if directory == 'asl-alphabet-test': continue
#     images = os.listdir(base_dir + '\..\input\asl-alphabet-test\' + directory')
#     dirl= '\..\input\asl-alphabet-test\' + directory + "\"
#     for image in dirl:
#         test_image = image.load_img(base_dir + '\..\input\asl-alphabet-test\' + directory + "\" + images[i], target_size=(img_width, img_height))
#         test_image = image.img_to_array(test_image)
#         test_image = np.expand_dims(test_image, axis=0)
#         result = model.predict(test_image)
#         pred.append(class_names[np.argmax(result)])
#         actual.append(directory)

In [ ]:
cm = confusion_matrix(y_true=actual, y_pred=pred)
plot_confusion_matrix(cm=cm, classes=class_names, title='Confusion Matrix')


In [ ]:
import keras
from tensorflow.keras.utils import load_img, img_to_array 
from keras.models import load_model
import PIL

img = load_img("/content/asl_alphabet_test/asl_alphabet_test/E_test.jpg")
img = img.resize((img_width, img_height))
img = img_to_array(img) 

img = img.reshape( -1,img_width, img_height,3)

print(img.shape)
model = load_model('model1.h5')
model.summary()
abc = model.predict(img)

print(abc)